### Clustering

In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

import time

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# import plotly.express as px


In [2]:
from kmodes.kprototypes import KPrototypes


* Code

In [3]:
invoice = pd.read_csv('data/invoice_refined.csv', low_memory=False)

In [50]:
inv_cl = invoice.copy()

In [51]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 41 columns):
Unnamed: 0             490789 non-null int64
Area / Locality        466918 non-null object
CITY                   490788 non-null object
Cash /Cashless Type    27306 non-null object
Claim No.              4797 non-null object
Cust Type              490789 non-null object
Customer No.           490789 non-null object
District               490789 non-null object
Expiry Date            22215 non-null object
Gate Pass Date         40851 non-null object
Gate Pass Time         490789 non-null object
Invoice Date           490789 non-null object
Invoice No             490789 non-null int64
Invoice Time           490789 non-null object
Job Card No            490789 non-null int64
JobCard Date           490789 non-null object
JobCard Time           490789 non-null object
KMs Reading            490789 non-null int64
Labour Total           490789 non-null float64
Make                   

In [52]:
invoice['KMs Reading'].isnull().sum()

0

In [53]:
invoice = invoice[invoice['Model'].notnull()]
invoice = invoice[invoice['KMs Reading']!=0]

In [54]:
km_cap = invoice[invoice['KMs Reading']>(300000)] #3 lakhs
# fig = px.histogram(km_cap, x='KMs Reading', barmode='overlay',marginal='box')
# fig.show()


In [55]:
km_cap[['Model','Make','KMs Reading']].sort_values(by='KMs Reading', ascending=False)

,Model,Make,KMs Reading
483740,PALIO STILE,FIAT,9999999
354505,PALIO STILE,FIAT,9999999
480246,PALIO STILE,FIAT,9999999
484789,CITY,HONDA,9949110
450769,ALTO,MARUTI SUZUKI,9804400
77385,OMNI,MARUTI SUZUKI,9632540
358583,OMNI,MARUTI SUZUKI,9632540
98269,OMNI,MARUTI SUZUKI,9632540
134395,XYLO,MAHINDRA & MAHINDRA,9104271
300548,MAHINDRA PIK UP,MAHINDRA & MAHINDRA,8947569


In [56]:
temp = invoice[invoice.Model=='PALIO STILE'][['Model','plant_state','KMs Reading','Total Value']]
temp1 = temp[temp['KMs Reading']<500000]['KMs Reading']
# fig = px.histogram(temp1, x='KMs Reading', nbins=100, barmode='overlay',marginal='box')
# fig.show()

In [57]:
mkmdmed = pd.DataFrame(invoice.groupby(['Make', 'Model'])['KMs Reading'].median().astype(int))
mkmdmed.reset_index(inplace=True)
mkmdmed

,Make,Model,KMs Reading
0,AUDI,A4,51187
1,AUDI,A6,47797
2,AUDI,A7,27042
3,AUDI,A8,56015
4,AUDI,Q5,29312
5,AUDI,Q7,67267
6,AUDI,TT,42400
7,BMW,3 SERIES,21735
8,BMW,5 SERIES,67260
9,BMW,6 SERIES,45021


In [58]:
indexes_gt_3L = km_cap.index
indexes_gt_3L

Int64Index([   318,    919,   1590,   1770,   1966,   1977,   2030,   2062,
              2068,   2135,
            ...
            491726, 491973, 491980, 491981, 491993, 492070, 492190, 492273,
            492277, 492289],
           dtype='int64', length=6191)

In [59]:
for i in indexes_gt_3L:
    i_model = inv_cl.loc[i,['Model']][0]
    i_make = inv_cl.loc[i,['Make']][0]
    val = mkmdmed[(mkmdmed['Model']==i_model)&(mkmdmed['Make']==i_make)]['KMs Reading'].iloc[0,]
    inv_cl.at[i,'KMs Reading'] = val
    

In [60]:
temp = invoice.groupby('plant_city')['Total Value'].count().astype(int).sort_values(ascending=False)
# print(temp)
# fig = px.histogram(x=temp.values, barmode='overlay',marginal='box', nbins=100)
# fig.show()

In [61]:
pcity_vc = pd.DataFrame(invoice.plant_city.value_counts())
# print(pcity_vcity_vc)
pcity_qcut5 = pd.qcut(pcity_vc['plant_city'],5, labels=['very low', 'low', 'medium', 'high', 'very high'])
pcity_qcut7 = pd.qcut(pcity_vc['plant_city'],7 ,
                       labels=['extremely low', 'very low', 'low', 'medium', 'high', 'very high', 'extremely high'])


In [62]:
pcity_qcut7.value_counts()

extremely high    28
very high         28
medium            28
very low          28
extremely low     28
high              27
low               27
Name: plant_city, dtype: int64

In [63]:
inv_cl['plant_city_qranges'] = inv_cl['plant_city'].map(pcity_qcut5.to_dict())

In [64]:
inv_cl['plant_city_qranges7'] = inv_cl['plant_city'].map(pcity_qcut7.to_dict())

In [65]:
temp = invoice.groupby('plant_state')['Total Value'].sum().astype(int).sort_values(ascending=False)
# fig = px.histogram(x=temp.values, barmode='overlay',marginal='box', nbins=10)
# fig.show()

In [66]:
inv_cl['plant_city_qranges'].value_counts(normalize=True)*100

very high    75.211751
high         14.946545
medium        6.436778
low           2.670394
very low      0.734532
Name: plant_city_qranges, dtype: float64

In [67]:
inv_cl['plant_city_qranges7'].value_counts(normalize=True)*100

extremely high    67.495808
very high         15.809442
high               8.043579
medium             4.658621
low                2.467252
very low           1.171583
extremely low      0.353716
Name: plant_city_qranges7, dtype: float64

In [68]:
#qcut

In [69]:
customer_no_freq_dict = inv_cl['Customer No.'].value_counts().to_dict()
inv_cl['Customer Frequency'] = inv_cl['Customer No.'].map(customer_no_freq_dict)

In [70]:
# inv_cl[['Customer No.','Customer Frequency']].head()

In [71]:
# cols for clustering: order type, month, city*, state (news?),
# kms reading, total value, frequency of visits basic or (level = make+model+city+state)

In [72]:
month_dict = {1: 'January',
              2: 'February',
              3: 'March',
              4: 'April',
              5: 'May',
              6: 'June',
              7: 'July',
              8: 'August',
              9: 'September',
              10: 'October',
              11: 'November',
              12: 'December'}
inv_cl['invoice_month_text'] = inv_cl['invoice_month'].map(month_dict)

In [73]:
inv_cl = inv_cl[['Customer No.','Order Type',
                 'plant_city', 'plant_state', 'invoice_month_text',
                 'KMs Reading', 'Total Value', 'plant_city_qranges7', 'Customer Frequency']]

In [74]:
X = inv_cl.drop(['Customer No.', 'plant_city_qranges7'],axis=1)

In [75]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490789 entries, 0 to 492313
Data columns (total 7 columns):
Order Type            490789 non-null object
plant_city            490789 non-null object
plant_state           490789 non-null object
invoice_month_text    490789 non-null object
KMs Reading           490789 non-null int64
Total Value           490789 non-null float64
Customer Frequency    490789 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 50.0+ MB


In [76]:
X.isnull().sum()

Order Type            0
plant_city            0
plant_state           0
invoice_month_text    0
KMs Reading           0
Total Value           0
Customer Frequency    0
dtype: int64

In [ ]:
# stop

In [80]:
num_clusters = 7
kproto = KPrototypes(n_clusters=num_clusters, init='Huang', random_state=42, max_iter=15, n_init=50) 
clusters = kproto.fit_predict(X, categorical=[0,1,2,3])
print(kproto.cost_)


KeyboardInterrupt: 

In [ ]:
# kproto

In [ ]:
# #Choosing optimal K value - after attaching clusters to main df
# cost = []
# for num_clusters in list(range(5,15)):
#     print(f'Computing for num of clusters = {num_clusters}')
#     kproto = KPrototypes(n_clusters=num_clusters, init='Huang', random_state=42, max_iter=15, n_init=50) 
#     kproto.fit_predict(X, categorical=[0,1,2,3])
#     cost.append(kproto.cost_)
#     print(f'Computation completed')

# plt.plot(cost)
# plt.xlabel('K')
# plt.ylabel('cost')
# plt.show